In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torchtext
from pathlib import Path
import matplotlib.pyplot as plt 
import sklearn.metrics as m
from transformers import BertTokenizer 
import warnings  
warnings.filterwarnings('ignore')
torch.__version__ , torchtext.__version__

In [ ]:
PATH = Path("/kaggle/input/nlp-disaster-tweets-eda")
os.listdir(PATH)

In [ ]:
#importamos los datos de entrenamientos

train = pd.read_csv(PATH/"train.csv")
train.info()

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
# Los transformers estan limitados en cuanto a la longitud de palabras a leer y es por ello que deberemos tenerlo en cuenta 

max_input_length = tokenizer.max_model_input_sizes["bert-base-uncased"] # Bert trabaja con 512.

def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2] #Deberemos restarle dos por los tokens unk y pad
    return tokens

In [ ]:
ID = torchtext.data.RawField()
KEYWORD = torchtext.data.RawField()
LOCATION = torchtext.data.RawField()
TEXT = torchtext.data.Field(batch_first=True,
                            use_vocab=False,
                            tokenize= tokenize_and_cut,
                            preprocessing = tokenizer.convert_tokens_to_ids,
                            init_token = tokenizer.cls_token_id,
                            eos_token = tokenizer.sep_token_id,
                            pad_token = tokenizer.pad_token_id,
                            unk_token = tokenizer.unk_token_id)
 
LABEL = torchtext.data.LabelField(dtype=torch.long)

dataset = torchtext.data.TabularDataset(
    path=PATH / 'train.csv',
    format = "CSV",
    fields = [("id",ID),("keyword",KEYWORD),("location",LOCATION),("text",TEXT),("target",LABEL)],
    skip_header=True
)

In [ ]:
len(dataset)

In [ ]:
ix=0
print(vars(dataset.examples[ix]))

In [ ]:
train_dataset, valid_dataset = dataset.split(
    split_ratio = 0.6,
    stratified =True,
    strata_field="target"
)
len(train_dataset), len(valid_dataset)

In [ ]:
# Proceso de tokenizacion
#MAX_VOCAB_SIZE = 10000
#TEXT.build_vocab(train_dataset, # construiremos nuestro vocabulario del tokenizador
#                 max_size = MAX_VOCAB_SIZE,
#                 vectors = "glove.6B.100d", # el parametro vectors nos permite obtener embedding ya entrenados
#                 unk_init = torch.Tensor.normal_) 
LABEL.build_vocab(train_dataset)

In [ ]:
len(LABEL.vocab)
# Esta limitado a 10000 pero ha creado dos tokens extra, uno es el unk que se lo pondrá a las palabras que no han entrado en nuestro diccionario por ser poco frecuentes, el segundo es el PAD que se trata de un token vacio que permite igualar las dimensiones de las frases

In [ ]:
# freqs nos devolvera los tokens con las veces que aparecen en nuestro datasets
TEXT.vocab.freqs.most_common(10)

In [ ]:
BATCH_SIZE = 64
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
dataloader = {
    "train": torchtext.data.BucketIterator(train_dataset, batch_size=BATCH_SIZE, shuffle=True, device=DEVICE),
    "val":  torchtext.data.BucketIterator(valid_dataset, batch_size=200, device=DEVICE) # Ponemos un mayor batch_size porque no tenemos que calcular gradientes y podremos ir mas rapido
    
}

## Modelo de LSTM cargando embeddings

In [ ]:
# CONSTRUIMOS RED NEURONAL
class LSTM(torch.nn.Module):
    def __init__(self, input_dim, embedding_dim=100, hidden_size=128, num_layers=2, n_outputs=2, bidirectional=False, dropout=0):
        super().__init__()
        self.embedding = torch.nn.Embedding(input_dim, embedding_dim) # Cogera una palabra y creará un vector que la represente
        self.rnn = torch.nn.LSTM( # Cargamos una red LSTM
            input_size= embedding_dim, 
            hidden_size = hidden_size,
            num_layers=num_layers,
            bidirectional=bidirectional,
            dropout= dropout)
        self.fc = torch.nn.Linear(2*hidden_size if bidirectional else hidden_size, n_outputs)
    
    def forward(self, text):
        embedded = self.embedding(text)#Le llega una frase un nos devolverá un vector de embedding_dim(128) dimensiones
        output, _ = self.rnn(embedded) # La capa de RNN nos devolvera los outputs y el valor del ultimo hidden state que no lo queremos para nada
        # Dimensiones en RNN y torchtext son ["longitud del texto","batch_size","hidden_size"]
        return self.fc(output[-1,:,:].squeeze(0)) # output[-1,:,:] Nos quedaremos con la ultima palabra, todo el batch_size y el hidden_size osea tenemos [1,64,128] dimensiones luego se lo pasaremos a la capa lineal quitandole la ultima palabra con la funcion squeeze(0)       

In [ ]:
# PROBAMOS NUESTRO MODELO
model = LSTM(input_dim=len(TEXT.vocab), bidirectional = False) # las dimensiones seran los 10002 palabras que tenemos en nuestro vocab
output = model ( torch.randint(0, len(TEXT.vocab), (100,64)))
output.shape

In [ ]:
# CREAMOS LA FUNCION PARA ENTRENAR NUESTRO MODELO

def fit(model, dataloader, epochs = 10, lr=1e-3):
    
    model.to(DEVICE) #Mandamos nuestro modelo a la gpu
    criterion = torch.nn.CrossEntropyLoss() # Funcion de perdida CrossEntropyLoss por que queremos mas de una salida sino seria BCE
    optimizer = torch.optim.Adam(model.parameters(), lr) # Optimizador con los parametros del modelo y añadimos nuestra lr
    
    hist = {"loss" : [], "f1": [], "val_loss":[],"val_f1":[]} # Guardaremos las metricas
    best_f1 = 0.
    for e in range(1, epochs+1):
        
        # ENTRENAMOS
        model.train() # Ponemos el model en modo entrenamiento
        l, f1s = [], []
        for batch in dataloader["train"]:
            optimizer.zero_grad() # Ponemos a cero los gradientes
            y_pred = model(batch.text) # Calculamos salida pasando al modelo el texto
            loss = criterion(y_pred, batch.target) # calulo loss functions
            l.append(loss.item()) # guardamos loss function
            loss.backward()#calculo gradiente
            optimizer.step()# actualizo los pesos
            y_pred= torch.argmax(y_pred, axis=1)# cogera el indice del valor mas grande, es decir dará 0 o 1
            f1s.append(m.f1_score(batch.target.cpu(), y_pred.cpu()))
        hist["loss"].append(np.mean(l))
        hist["f1"].append(np.mean(f1s))
        
        #EVALUAMOS
        model.eval()
        l,acc,f1s = [],[],[]
        with torch.no_grad():
            for batch in dataloader["val"]:
                y_pred = model(batch.text)
                loss = criterion (y_pred, batch.target)
                l.append(loss.item())
                y_pred = torch.argmax(y_pred, axis=1)
                f1s.append(m.f1_score(batch.target.cpu(), y_pred.cpu())) # Si trabajas con el paquete sklearn los datos deben estar en la cpu y en formato numpy
        hist["val_loss"].append(np.mean(l))
        hist["val_f1"].append(np.mean(f1s))
        # CALLBACKS SAVE BEST MODEL
        if hist["val_f1"][-1] > best_f1:
            best_f1 = hist["val_f1"][-1]
            torch.save(model.state_dict(),"ckpt.pt")
        print(f'Epoch {e}/{epochs} loss:{hist["loss"][-1]:.5f} f1:{hist["f1"][-1]:.5f} val_loss:{hist["val_loss"][-1]:.5f} val_f1:{hist["val_f1"][-1]:.5f}')
    model.load_state_dict(torch.load("ckpt.pt"))
    return hist

In [ ]:
# INSTANCIAMOS LA RNN Y ENTRENAMOS
model = LSTM(input_dim=len(TEXT.vocab))

pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)# Copiamos los embedding descargados y los metemos en nuestro modelo
model.embedding.weight.data[TEXT.vocab.stoi[TEXT.unk_token]]=torch.zeros(100)
model.embedding.weight.data[TEXT.vocab.stoi[TEXT.pad_token]]=torch.zeros(100)


hist = fit(model, dataloader)

In [ ]:
def plot(hist): # Funcion para graficar nuestras metricas 
    fig = plt.figure(dpi = 200, figsize= (10,3))
    ax = plt.subplot(121)
    hist = pd.DataFrame(hist)
    hist[["loss","val_loss"]].plot(ax=ax, grid=True)
    ax = plt.subplot(122)
    hist[["f1", "val_f1"]].plot(ax=ax, grid=True)
    plt.show()
plot(hist)

Estos malos resultados es porque las RNN sencillas no funcionan bien con longuitudes grandes. Van bien con secuancias de texto de 10-20 pero en nuestro caso podemos tener unas secuencias mayor a 100, entonces estas redes tan sencillas fallan mucho. Deberemos buscar mejores modelos como LSTM, bidireccionales o transformers

## TEST

In [ ]:
# Creamos nuestros dataset de test
test_dataset = torchtext.data.TabularDataset(
    path=PATH/'test.csv',
    format = "CSV",
    fields = [("id",ID),("keyword",KEYWORD),("location",LOCATION),("text",TEXT)],
    skip_header=True
)
len(test_dataset)

In [ ]:
# Comprobamos si es testo correcto
ix=3258
print(vars(test_dataset.examples[ix]))

In [ ]:
test_dataloader = torchtext.data.BucketIterator(test_dataset, batch_size=BATCH_SIZE, shuffle=False, device=DEVICE)

In [ ]:
def predict():
    model.eval()
    preds = torch.tensor([]).to(DEVICE)
    with torch.no_grad():
        for batch in test_dataloader:
            y_pred=model(batch.text)
            y_pred = torch.argmax(y_pred, axis=1)
            preds = torch.cat([preds, y_pred])
    return preds

In [ ]:
preds = predict()
preds

In [ ]:
submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
submission.target = preds.cpu().long()# Me traigo mis pres a la cpu 
submission.to_csv("submission.csv", index=False)